In [1]:
import os
import csv
import time
import datetime
import random
import json

from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    numFilters = 128

    filterSizes = [2, 3, 4, 5]
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:130: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 构建模型
class TextCNN(object):
    """
    Text CNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            # 卷积的输入是思维[batch_size, width, height, channel]，因此需要增加维度，用tf.expand_dims来增大维度
            self.embeddedWordsExpanded = tf.expand_dims(self.embeddedWords, -1)

        # 创建卷积和池化层
        pooledOutputs = []
        # 有三种size的filter，3， 4， 5，textCNN是个多通道单层卷积的模型，可以看作三个单层的卷积模型的融合
        for i, filterSize in enumerate(config.model.filterSizes):
            with tf.name_scope("conv-maxpool-%s" % filterSize):
                # 卷积层，卷积核尺寸为filterSize * embeddingSize，卷积核的个数为numFilters
                # 初始化权重矩阵和偏置
                filterShape = [filterSize, config.model.embeddingSize, 1, config.model.numFilters]
                W = tf.Variable(tf.truncated_normal(filterShape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[config.model.numFilters]), name="b")
                conv = tf.nn.conv2d(
                    self.embeddedWordsExpanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                
                # relu函数的非线性映射
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                # 池化层，最大池化，池化是对卷积后的序列取一个最大值
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, config.sequenceLength - filterSize + 1, 1, 1],  # ksize shape: [batch, height, width, channels]
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooledOutputs.append(pooled)  # 将三种size的filter的输出一起加入到列表中

        # 得到CNN网络的输出长度
        numFiltersTotal = config.model.numFilters * len(config.model.filterSizes)
        
        # 池化后的维度不变，按照最后的维度channel来concat
        self.hPool = tf.concat(pooledOutputs, 3)
        
        # 摊平成二维的数据输入到全连接层
        self.hPoolFlat = tf.reshape(self.hPool, [-1, numFiltersTotal])

        # dropout
        with tf.name_scope("dropout"):
            self.hDrop = tf.nn.dropout(self.hPoolFlat, self.dropoutKeepProb)
       
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[numFiltersTotal, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(self.hDrop, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [8]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        cnn = TextCNN(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(cnn.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", cnn.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/textCNN/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              cnn.inputX: batchX,
              cnn.inputY: batchY,
              cnn.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, cnn.loss, cnn.predictions, cnn.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              cnn.inputX: batchX,
              cnn.inputY: batchY,
              cnn.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, cnn.loss, cnn.predictions, cnn.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/textCNN/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        inputs = {"inputX": tf.saved_model.utils.build_tensor_info(cnn.inputX),
                  "keepProb": tf.saved_model.utils.build_tensor_info(cnn.dropoutKeepProb)}

        outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(cnn.binaryPreds)}

        prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
                                                                                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
                                            signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

        builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/hist is illegal; using conv-maxpool-2/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/sparsity is illegal; using conv-maxpool-2/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/hist is illegal; using conv-maxpool-2/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/sparsity is illegal; using conv-maxpool-2/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using 

2018-12-26T18:22:06.351388, step: 47, loss: 1.1892750263214111, acc: 0.5469, auc: 0.5813, precision: 0.48, recall: 0.6545
2018-12-26T18:22:06.505589, step: 48, loss: 0.9242684841156006, acc: 0.6484, auc: 0.6821, precision: 0.678, recall: 0.6061
2018-12-26T18:22:06.653700, step: 49, loss: 1.0500402450561523, acc: 0.5859, auc: 0.6112, precision: 0.6207, recall: 0.5373
2018-12-26T18:22:06.813964, step: 50, loss: 1.200317144393921, acc: 0.5156, auc: 0.5431, precision: 0.48, recall: 0.4
2018-12-26T18:22:06.962367, step: 51, loss: 1.1802515983581543, acc: 0.5703, auc: 0.5512, precision: 0.5641, recall: 0.3667
2018-12-26T18:22:07.111723, step: 52, loss: 1.2376327514648438, acc: 0.5547, auc: 0.549, precision: 0.5106, recall: 0.4138
2018-12-26T18:22:07.260346, step: 53, loss: 1.5337551832199097, acc: 0.4297, auc: 0.4303, precision: 0.4091, recall: 0.2769
2018-12-26T18:22:07.418023, step: 54, loss: 1.1761348247528076, acc: 0.5078, auc: 0.522, precision: 0.4884, recall: 0.3387
2018-12-26T18:22:07

2018-12-26T18:22:22.291422, step: 112, loss: 0.6048027276992798, acc: 0.7031, auc: 0.7848, precision: 0.7586, recall: 0.6471
2018-12-26T18:22:22.448718, step: 113, loss: 0.5197864174842834, acc: 0.7422, auc: 0.8292, precision: 0.7797, recall: 0.697
2018-12-26T18:22:22.599481, step: 114, loss: 0.60899817943573, acc: 0.6641, auc: 0.7582, precision: 0.6613, recall: 0.6508
2018-12-26T18:22:22.751757, step: 115, loss: 0.6150535345077515, acc: 0.6719, auc: 0.7664, precision: 0.7755, recall: 0.5507
2018-12-26T18:22:22.897501, step: 116, loss: 0.6705420613288879, acc: 0.6094, auc: 0.7289, precision: 0.6833, recall: 0.5694
2018-12-26T18:22:23.054478, step: 117, loss: 0.5960569977760315, acc: 0.7031, auc: 0.7768, precision: 0.7931, recall: 0.6389
2018-12-26T18:22:23.211213, step: 118, loss: 0.5037750005722046, acc: 0.7266, auc: 0.8266, precision: 0.791, recall: 0.7162
2018-12-26T18:22:23.361339, step: 119, loss: 0.6424188613891602, acc: 0.7109, auc: 0.7717, precision: 0.7049, recall: 0.6935
2018

2018-12-26T18:22:32.419352, step: 178, loss: 0.47027653455734253, acc: 0.7656, auc: 0.874, precision: 0.925, recall: 0.5781
2018-12-26T18:22:32.575397, step: 179, loss: 0.4400181174278259, acc: 0.7734, auc: 0.8949, precision: 0.898, recall: 0.6471
2018-12-26T18:22:32.725227, step: 180, loss: 0.35575369000434875, acc: 0.8203, auc: 0.9239, precision: 0.9107, recall: 0.7391
2018-12-26T18:22:32.883190, step: 181, loss: 0.5393904447555542, acc: 0.7656, auc: 0.8276, precision: 0.7708, recall: 0.6607
2018-12-26T18:22:33.036049, step: 182, loss: 0.43009111285209656, acc: 0.75, auc: 0.8818, precision: 0.7778, recall: 0.7313
2018-12-26T18:22:33.186172, step: 183, loss: 0.4205598831176758, acc: 0.8125, auc: 0.9154, precision: 0.7571, recall: 0.8833
2018-12-26T18:22:33.344071, step: 184, loss: 0.3894326090812683, acc: 0.8281, auc: 0.8881, precision: 0.8734, recall: 0.8519
2018-12-26T18:22:33.494771, step: 185, loss: 0.41712504625320435, acc: 0.7969, auc: 0.8877, precision: 0.8276, recall: 0.75
201

2018-12-26T18:22:48.258283, step: 242, loss: 0.36950287222862244, acc: 0.8047, auc: 0.9138, precision: 0.8276, recall: 0.7619
2018-12-26T18:22:48.412907, step: 243, loss: 0.4091613292694092, acc: 0.8125, auc: 0.9035, precision: 0.9318, recall: 0.6613
2018-12-26T18:22:48.568704, step: 244, loss: 0.43924424052238464, acc: 0.8047, auc: 0.8821, precision: 0.8545, recall: 0.7344
2018-12-26T18:22:48.727596, step: 245, loss: 0.40640804171562195, acc: 0.7656, auc: 0.9062, precision: 0.8864, recall: 0.6094
2018-12-26T18:22:48.886979, step: 246, loss: 0.4484177231788635, acc: 0.7812, auc: 0.8667, precision: 0.7647, recall: 0.7091
2018-12-26T18:22:49.045430, step: 247, loss: 0.5184038877487183, acc: 0.7891, auc: 0.8608, precision: 0.8364, recall: 0.7188
2018-12-26T18:22:49.196181, step: 248, loss: 0.3007338047027588, acc: 0.8203, auc: 0.9451, precision: 0.9123, recall: 0.7429
2018-12-26T18:22:49.355311, step: 249, loss: 0.4071643650531769, acc: 0.8203, auc: 0.8988, precision: 0.8421, recall: 0.77

...

2018-12-26T18:26:21.291861, step: 1220, loss: 0.0455472357571125, acc: 0.9922, auc: 1.0, precision: 0.9841, recall: 1.0
2018-12-26T18:26:21.446809, step: 1221, loss: 0.04718613252043724, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9844
2018-12-26T18:26:21.605393, step: 1222, loss: 0.044151660054922104, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9833
2018-12-26T18:26:21.757071, step: 1223, loss: 0.06043483316898346, acc: 0.9922, auc: 0.9983, precision: 1.0, recall: 0.9846
2018-12-26T18:26:21.907178, step: 1224, loss: 0.10588078945875168, acc: 0.9688, auc: 0.9945, precision: 0.9726, recall: 0.9726
2018-12-26T18:26:22.067666, step: 1225, loss: 0.04411960393190384, acc: 0.9844, auc: 0.9998, precision: 0.9821, recall: 0.9821
2018-12-26T18:26:22.223569, step: 1226, loss: 0.06544626504182816, acc: 0.9844, auc: 0.9983, precision: 0.9861, recall: 0.9861
2018-12-26T18:26:22.410143, step: 1227, loss: 0.05658412724733353, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9844
2018-12-26T

2018-12-26T18:26:31.965060, step: 1288, loss: 0.038109198212623596, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:26:32.126678, step: 1289, loss: 0.04725267365574837, acc: 0.9922, auc: 1.0, precision: 0.9831, recall: 1.0
2018-12-26T18:26:32.282290, step: 1290, loss: 0.04129287973046303, acc: 0.9844, auc: 0.9998, precision: 0.9836, recall: 0.9836
2018-12-26T18:26:32.438870, step: 1291, loss: 0.033470869064331055, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:26:32.605175, step: 1292, loss: 0.0411866120994091, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9859
2018-12-26T18:26:32.759254, step: 1293, loss: 0.025071196258068085, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9848
2018-12-26T18:26:32.914655, step: 1294, loss: 0.03350221365690231, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9833
2018-12-26T18:26:33.070988, step: 1295, loss: 0.04241412132978439, acc: 0.9766, auc: 1.0, precision: 1.0, recall: 0.9559
2018-12-26T18:26:33.226409, step: 12

2018-12-26T18:26:48.645649, step: 1356, loss: 0.02059594914317131, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:26:48.805176, step: 1357, loss: 0.043201744556427, acc: 0.9922, auc: 0.9988, precision: 1.0, recall: 0.9855
2018-12-26T18:26:48.965942, step: 1358, loss: 0.032396040856838226, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9853
2018-12-26T18:26:49.135467, step: 1359, loss: 0.09957434237003326, acc: 0.9688, auc: 0.9946, precision: 0.9538, recall: 0.9841
2018-12-26T18:26:49.293683, step: 1360, loss: 0.019990889355540276, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:26:49.446589, step: 1361, loss: 0.03721063956618309, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:26:49.601517, step: 1362, loss: 0.027115436270833015, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9853
2018-12-26T18:26:49.754529, step: 1363, loss: 0.01843993179500103, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9836
2018-12-26T18:26:49.910805, step: 1364, 

2018-12-26T18:27:05.486299, step: 1424, loss: 0.02464735321700573, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:05.639290, step: 1425, loss: 0.02520758844912052, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:05.800130, step: 1426, loss: 0.021770808845758438, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:05.954624, step: 1427, loss: 0.02049967087805271, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:06.107507, step: 1428, loss: 0.045311495661735535, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:06.260751, step: 1429, loss: 0.028551999479532242, acc: 0.9844, auc: 1.0, precision: 1.0, recall: 0.9718
2018-12-26T18:27:06.414026, step: 1430, loss: 0.02290562354028225, acc: 0.9922, auc: 1.0, precision: 0.9867, recall: 1.0
2018-12-26T18:27:06.568088, step: 1431, loss: 0.021933987736701965, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:06.721246, step: 1432, loss: 0.0189948361366987

2018-12-26T18:27:16.488419, step: 1494, loss: 0.02260580286383629, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:16.641383, step: 1495, loss: 0.04864628612995148, acc: 0.9688, auc: 1.0, precision: 1.0, recall: 0.9437
2018-12-26T18:27:16.790607, step: 1496, loss: 0.024386420845985413, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9851
2018-12-26T18:27:16.941393, step: 1497, loss: 0.02363758161664009, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:17.094155, step: 1498, loss: 0.019330209121108055, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-26T18:27:17.246800, step: 1499, loss: 0.03760554641485214, acc: 0.9922, auc: 1.0, precision: 0.9839, recall: 1.0
2018-12-26T18:27:17.399463, step: 1500, loss: 0.04160323739051819, acc: 0.9922, auc: 0.9993, precision: 1.0, recall: 0.9844

Evaluation:
2018-12-26T18:27:23.287043, step: 1500, loss: 0.39950417648804815, acc: 0.861636842105263, auc: 0.9332026315789472, precision: 0.8726105263157896, recall